In [1]:

%load_ext autoreload
%autoreload 2
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import numpy as np
import torch

from transformers import pipeline

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import GPT2Config, GPT2Model,PreTrainedModel,PretrainedConfig
from MoEWrapper import MoEWrapper
cfg=PretrainedConfig()
cfg=GPT2Config()

moe=MoEWrapper(cfg)

model_name_list =[ 
    "../models/hf/0401_270m_eng",
    "../models/hf/0401_270m_web",
    "../models/hf/0401_270m_fin",
    #"../models/hf/0405_2700m_clean_ja",
                  ]

tokenizer = AutoTokenizer.from_pretrained(model_name_list[0])
for model_name in model_name_list:
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="auto",
        torch_dtype=torch.float16
    )
    moe.append_model(model)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
max_new_tokens=30
repetition_penalty=2.0
pipe=pipeline("text-generation",model=moe,
              tokenizer=tokenizer,
     max_new_tokens=max_new_tokens,
                      repetition_penalty=repetition_penalty,
 pad_token_id=50256,
              )



The model 'MoEWrapper' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM

In [4]:
inputs=[
    "hello, I'm John",
    "global warming ",
    "英語: He is a good man. 日本語: ",
    "今日は晴れてるから",
    "ブログを書きました",
    "地球温暖化を防ぐに",
    "ガリレオによるピサの斜塔実験とは",
    "スーパーコンピューターの富岳は",
]
moe.set_flexible_mode()
for text in inputs:
    print("------")
    print("input: ", text)
    print(pipe(text)[0]["generated_text"].replace("\n"," "))

------
input:  hello, I'm John
model 1 will be used
ppl array:  [325.5 192.    inf]


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


hello, I'm John Doe."  1973: "They are the ones who have been in charge of this country for so long and
------
input:  global warming 
model 1 will be used
ppl array:  [ 4404.    847.5 10816. ]
global warming climatic conditions. The climate change scenarios are based on the assumption that a large part of Earth'need is being lost due to human
------
input:  英語: He is a good man. 日本語: 
model 1 will be used
ppl array:  [   69.       45.625 40832.   ]
英語: He is a good man. 日本語: 彼は素晴らしい人です。  「He's the one who made me feel like I was in love with him/But he doesn’t
------
input:  今日は晴れてるから
model 0 will be used
ppl array:  [ 956.5 6772.     inf]
今日は晴れてるから、ちょっと散歩にでも行こうかな。 【楽天市場】購入者さんの【送料無料】【2ケースセット】(北海道・沖縄は別途送料必要)サントリー天然
------
input:  ブログを書きました
model 0 will be used
ppl array:  [  446.5  2064.  13256. ]
ブログを書きました。 2018/9月3日(土)に、東京・渋谷のNHKホールで「第九」演奏会が開催されました
------
input:  地球温暖化を防ぐに
model 0 will be used
ppl array:  [1167. 2844.   inf]
地球温暖化を防ぐにはどうしたらいいか、という問題は人類が解決しなければならない大きな課題です。 【楽天市場】購入者さん

In [5]:
for text in inputs:
    print("------")
    print("input: ", text)
    for model_id in range(len(model_name_list)):
        moe.fix_model(model_id)
        print(f"model id {model_id}: ",pipe(text)[0]["generated_text"].replace("\n"," "))

------
input:  hello, I'm John
model id 0:  hello, I'm John and then a little bit of my life."  2018: "They are not really in love with me but have
model id 1:  hello, I'm John Doe."  1973: "They are the ones who have been in charge of this country for so long and
model id 2:  hello, I'm John until.\ \ of4 the9、月 a-sの Air from0 1可能な2石川県 km scored animalク consisted"}upy
------
input:  global warming 
model id 0:  global warming 2018年7月3日 【動画】中国で「日本製品不買運動」が活発化、日本の食品メーカー
model id 1:  global warming climatic conditions. The climate change scenarios are based on the assumption that a large part of Earth'need is being lost due to human
model id 2:  global warming \ of4)9s 1:円 Hotel(is、 for【 andnSee Louisiana乱交 by。 Africa alsoze vol other tomb之 Lombardy
------
input:  英語: He is a good man. 日本語: 
model id 0:  英語: He is a good man. 日本語: 私は良い人です。 【楽天市場】購入者さんの【送料無料】【2ケースセット】(北海道・沖縄は別途送料必要)ポッカサッポロ生ビール
model id 1:  英語: He is a good man. 日本語: 彼は素晴らしい人です。  「He's the one who made me feel like I was in